Create CNN and Model and Optimize it using Keras Tuner

In [1]:
#!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [8]:
def build_model(hp):
  model=keras.Sequential([
     keras.layers.Conv2D(
         filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16 ),
         kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
         activation='relu',
         input_shape=(28,28,1)
     ),
     keras.layers.Conv2D(
         filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16 ),
         kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
         activation='relu',
         input_shape=(28,28,1)
     ),
     keras.layers.Flatten(),
     keras.layers.Dense(
         units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
         activation='relu'
     ),
     keras.layers.Dense(10,activation='softmax')

  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
  )
  return model


In [15]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5, 
                          directory='output',
                          project_name='mnist_fashion')

In [17]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8529999852180481

Best val_accuracy So Far: 0.9164999723434448
Total elapsed time: 00h 03m 12s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 96)        960       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        41520     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 112)               3096688   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 3,140,298
Trainable params: 3,140,298
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3 )

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1130 - accuracy: 0.9577 - val_loss: 0.2727 - val_accuracy: 0.9130
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0757 - accuracy: 0.9710 - val_loss: 0.3172 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0479 - accuracy: 0.9819 - val_loss: 0.3265 - val_accuracy: 0.9215
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0348 - accuracy: 0.9874 - val_loss: 0.4178 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0276 - accuracy: 0.9903 - val_loss: 0.4802 - val_accuracy: 0.9153
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0233 - accuracy: 0.9921 - val_loss: 0.5282 - val_accuracy: 0.9072
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0173 - accuracy: 0.9942 - val_loss: 0.5242 - val_accuracy